In [ ]:
#### TENSORFLOW BEGIN
import tensorflow as tf
import pandas as pd
from tensorflow.keras import layers
import numpy as np

In [ ]:
data = pd.read_pickle("big_framedata.pkl")

data['label'] = pd.factorize(data['label'])[0] + 1
features = data.drop('label', axis=1)
labels = data['label']

In [ ]:
features_tensor = tf.convert_to_tensor(features.values, dtype=tf.float32)
labels_tensor = tf.convert_to_tensor(labels.values, dtype=tf.int32)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features_tensor, labels_tensor))

In [ ]:
def create_model(input_size, num_classes):
    model = tf.keras.Sequential([
        layers.Input(shape=(input_size,)),  # Replace 'input_shape' with the shape of your input data
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(num_classes)  # Replace 'num_classes' with the number of output classes
    ])
    return model

In [ ]:
model = create_model(30, 5)


# Step 2: Specify the loss function and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

# Step 3: Create the training loop
batch_size = 32
dataset = dataset.batch(batch_size)

epochs = 5 # You can adjust the number of epochs based on your training needs
losses = []
# Iterate over epochs
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    
    # Initialize the average loss metric
    avg_loss = tf.keras.metrics.Mean()
    
    # Iterate over batches
    for batch_features, batch_labels in dataset:
        # Open a GradientTape to record the operations for automatic differentiation
        with tf.GradientTape() as tape:
            # Perform forward pass
            predictions = model(batch_features, training=True)
            # Calculate the loss
            loss = loss_object(batch_labels, predictions)
        
        # Calculate gradients
        gradients = tape.gradient(loss, model.trainable_variables)
        # Apply gradients to update the model parameters
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        # Update the average loss metric
        avg_loss.update_state(loss)
    
    # Print the average loss for the epoch
    print(f"Average Loss: {avg_loss.result().numpy()}")
    losses.append(avg_loss.result().numpy())

In [ ]:
tf.keras.models.save_model(model, "2023-07-16_model.h5")

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_activations=True, expand_nested=True)

In [ ]:
#model.predict()
row = big_frame.iloc[1009][1:]
list(row)
tf_row = tf.convert_to_tensor(row, dtype=tf.int32)

In [ ]:
#print(dir(model))
model.weights